In [ ]:
#please run twice
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9b21f39/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [ ]:
!apt-get install -qq xvfb libgl1-mesa-glx
!pip install pyvista -qq
import pyvista
pyvista.global_theme.jupyter_backend = 'static'
pyvista.global_theme.notebook = True

In [ ]:
!pip install pyvistaqt
!pip install pyqt5-tools

In [4]:
import numpy as np
from petsc4py import PETSc
assert not np.issubdtype(PETSc.ScalarType, np.complexfloating)
import numpy as np
import pyvista
from dolfinx.fem import (Constant, dirichletbc, Function, FunctionSpace, assemble_scalar,
                         form, locate_dofs_geometrical, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile
from dolfinx.mesh import create_unit_square, locate_entities
from dolfinx.plot import create_vtk_mesh
from ufl import (SpatialCoordinate, TestFunction, TrialFunction,
                 dx, grad, inner)

from mpi4py import MPI
from petsc4py.PETSc import ScalarType

pyvista.start_xvfb()
import ufl

ThermoElasticity

In [5]:
domain = create_unit_square(MPI.COMM_WORLD, 10, 10)

In [6]:
u_1 = ufl.VectorElement("CG", domain.ufl_cell(), 1)
t_1 = ufl.FiniteElement("CG", domain.ufl_cell(), 2)
mixed_elem=u_1*t_1

V = FunctionSpace(domain, mixed_elem)

U_ = TestFunction(V)
(u_, Theta_) = ufl.split(U_)
dU = TrialFunction(V)
(du, dTheta) = ufl.split(dU)

In [7]:
# boundary markers
def sides(x):
    return np.logical_or(np.isclose(x[0],0), np.isclose(x[0],1))

def bound(x):
    a=np.logical_or(np.isclose(x[0],0), np.isclose(x[0],1))
    b=np.logical_or(np.isclose(x[1],0), np.isclose(x[1],1))
    return np.logical_or(a, b)

sides_facets = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, sides)
all_facets = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, bound)

In [8]:
V0, submap0 = V.sub(0).collapse()
V1, submap = V.sub(1).collapse()
 # Define block boundary conditions
bdofs_D = dolfinx.fem.locate_dofs_topological(V0, domain.topology.dim - 1, sides_facets)
bdofs_T = dolfinx.fem.locate_dofs_topological(V1, domain.topology.dim - 1, all_facets)

u_0 = np.array((0,) * domain.geometry.dim, dtype=PETSc.ScalarType)


bc2 = dolfinx.fem.dirichletbc(PETSc.ScalarType(0), bdofs_T, V1)

bc1 = dolfinx.fem.dirichletbc(u_0, bdofs_D, V0)

In [9]:
def eps(v):
    return ufl.sym(grad(v))


def sigma(v, Theta):
    return (ufl.tr(eps(v)) - Theta)*ufl.Identity(2) + 2*eps(v)

In [10]:
f=Constant(domain, ScalarType((0, 0)))
f1=Constant(domain, ScalarType((1)))

In [11]:

therm_a = ufl.dot(grad(Theta_),grad(dTheta))*dx
mech_a = inner(sigma(du, dTheta), eps(u_))*dx

a=mech_a+therm_a

L=ufl.dot(f,du)*dx+f1*dTheta*dx

problem = LinearProblem(a, L, bcs=[bc1, bc2], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

Cahn-Hilliard Equation:

https://docs.fenicsproject.org/dolfinx/v0.6.0/python/demos/demo_cahn-hilliard.html

In [12]:
import os

import numpy as np

import ufl
from dolfinx import log, plot
from dolfinx.fem import Function, FunctionSpace
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, create_unit_square
from dolfinx.nls.petsc import NewtonSolver
from ufl import dx, grad, inner

from mpi4py import MPI
from petsc4py import PETSc

try:
    import pyvista as pv
    import pyvistaqt as pvqt
    have_pyvista = True
    if pv.OFF_SCREEN:
        pv.start_xvfb(wait=0.5) 
except ModuleNotFoundError:
    print("pyvista and pyvistaqt are required to visualise the solution")
    have_pyvista = False
 
# Save all logging to file
log.set_output_file("log.txt")

In [13]:
have_pyvista=False

In [14]:
lmbda = 1.0e-02  # surface parameter
dt = 5.0e-06  # time step
theta = 0.5  # time stepping family, e.g. theta=1 -> backward Euler, theta=0.5 -> Crank-Nicholson

In [15]:
msh = create_unit_square(MPI.COMM_WORLD, 96, 96, CellType.triangle)
P1 = ufl.FiniteElement("Lagrange", msh.ufl_cell(), 1)
ME = FunctionSpace(msh, P1 * P1)

In [16]:
q, v = ufl.TestFunctions(ME)

In [17]:
u = Function(ME)  # current solution
u0 = Function(ME)  # solution from previous converged step

# Split mixed functions
c, mu = ufl.split(u)
c0, mu0 = ufl.split(u0)

In [18]:
# Zero u
u.x.array[:] = 0.0

# Interpolate initial condition
u.sub(0).interpolate(lambda x: 0.63 + 0.02 * (0.5 - np.random.rand(x.shape[1])))
u.x.scatter_forward()

In [19]:
# Compute the chemical potential df/dc
c = ufl.variable(c)
f = 100 * c**2 * (1 - c)**2
dfdc = ufl.diff(f, c)

In [20]:
# mu_(n+theta)
mu_mid = (1.0 - theta) * mu0 + theta * mu

In [21]:
# Weak statement of the equations
F0 = inner(c, q) * dx - inner(c0, q) * dx + dt * inner(grad(mu_mid), grad(q)) * dx
F1 = inner(mu, v) * dx - inner(dfdc, v) * dx - lmbda * inner(grad(c), grad(v)) * dx
F = F0 + F1

In [22]:
#Create nonlinear problem and Newton solver
problem = NonlinearProblem(F, u)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6

# We can customize the linear solver used inside the NewtonSolver by
# modifying the PETSc options
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
ksp.setFromOptions()

In [ ]:
#Output file
file = XDMFFile(MPI.COMM_WORLD, "demo_ch/output.xdmf", "w")
file.write_mesh(msh)

# Step in time
t = 0.0

#  Reduce run time if on test (CI) server
if "CI" in os.environ.keys() or "GITHUB_ACTIONS" in os.environ.keys():
    T = 3 * dt
else:
    T = 50 * dt

# Get the sub-space for c and the corresponding dofs in the mixed space
# vector
V0, dofs = ME.sub(0).collapse()

# Prepare viewer for plotting the solution during the computation
if have_pyvista:
    # Create a VTK 'mesh' with 'nodes' at the function dofs
    topology, cell_types, x = plot.create_vtk_mesh(V0)
    grid = pv.UnstructuredGrid(topology, cell_types, x)

    # Set output data
    grid.point_data["c"] = u.x.array[dofs].real
    grid.set_active_scalars("c")

    p = pvqt.BackgroundPlotter(title="concentration", auto_update=True)
    p.add_mesh(grid, clim=[0, 1])
    p.view_xy(True)
    p.add_text(f"time: {t}", font_size=12, name="timelabel")

c = u.sub(0)
u0.x.array[:] = u.x.array
while (t < T):
    t += dt
    r = solver.solve(u)
    print(f"Step {int(t/dt)}: num iterations: {r[0]}")
    u0.x.array[:] = u.x.array
    file.write_function(c, t)

    # Update the plot window
    if have_pyvista:
        p.add_text(f"time: {t:.2e}", font_size=12, name="timelabel")
        grid.point_data["c"] = u.x.array[dofs].real
        p.app.processEvents()

file.close()

# Update ghost entries and plot
if have_pyvista:
    u.x.scatter_forward()
    grid.point_data["c"] = u.x.array[dofs].real
    screenshot = None
    if pv.OFF_SCREEN:
        screenshot = "c.png"
    pv.plot(grid, show_edges=True, screenshot=screenshot)